First, move within a simulation directory. This directory should contains the sub-directory 'output/' that contains snapshot files, and a file 'snapshot_times.txt' that lists the scale-factors, redshifts, times, and indices of all snapshots stored from the simulation.

Ensure that gizmo_analysis and utilities directories are in your python path, then...

In [3]:
import gizmo_analysis as gizmo
import utilities as ut

import numpy as np

ImportError: attempted relative import with no known parent package

In [2]:
# you can access the files as named or use the aliases in __init__.py to keep it shorter 
# for example, these are the same:

gizmo.gizmo_io
gizmo.io

NameError: name 'gizmo' is not defined

# read particle data

In [ ]:
# read star and dark-matter particles at z = 0

part = gizmo.io.Read.read_snapshots(['star', 'dark'], 'redshift', 0)

In [ ]:
# alternately, read all particle species at z = 0

part = gizmo.io.Read.read_snapshots('all', 'redshift', 0)

In [ ]:
# this tutorial assumes that you are in the root directory of a simulation, 
# but you can read a simulation at any location using input argument 'simulation_directory'

part = gizmo.io.Read.read_snapshots('all', 'redshift', 0, simulation_directory='/arb/it/rary/loc/a/tion')

In [ ]:
# each particle species is stored as its own dictionary
# 'star' = stars, 'gas' = gas, 'dark' = dark matter,
# 'dark2', 'dark3', 'dark4', etc = low-resolution dark matter (but you rarely will be interested in this)

part.keys()

In [ ]:
# properties of star particles are stored via dictionary

part['star'].keys()

In [ ]:
# properties of dark-matter particles

part['dark'].keys()

In [ ]:
# properties of gas particles

part['gas'].keys()

# particle properties

In [ ]:
# 3-D position of star particles (particle number x dimension number) [kpc comoving]

part['star']['position']

In [ ]:
# 3-D velocity of star particle (particle number x dimension number) [km / s]

part['star']['velocity']

In [ ]:
# mass of star particle [Msun]

part['star']['mass']

In [ ]:
# formation scale-factor of star particle

part['star']['form.scalefactor']

In [ ]:
# use .prop() to compute derived quantities,
# such as the time [Gyr] when a star particle formed
# see gizmo.io.ParticleDictionaryClass for all options for derived quantities

part['star'].prop('form.time')

In [ ]:
# similarly, get the age of a star particle (the lookback time to when it formed) [Gyr]

part['star'].prop('age')

# elemental abundances / metallicities

In [ ]:
# elemental abundance is stored in the catalog as linear *mass fraction*
# one value for each element, in a array (particle_number x element number)
# the first value is the mass fraction of all metals (everything not H, He)
# then He, C, N, O, etc

part['star']['massfraction']

In [ ]:
# get individual elements by their index

# total metal mass fraction (everything not H, He) is index 0
print(part['star']['massfraction'][:, 0])

# iron is index 10
print(part['star']['massfraction'][:, 10])

In [ ]:
# alternately use .prop() to compute derived quantities,
# including calling element by its name or symbol
# see gizmo.io.ParticleDictionaryClass for all options for derived quantities

print(part['star'].prop('massfraction.metals'))
print(part['star'].prop('massfraction.carbon'))
print(part['star'].prop('massfraction.iron'))
print(part['star'].prop('massfraction.fe'))  # can use name or symbol

In [ ]:
# also use .prop() to compute metallicity [Z / H]
# for example, iron abundance [Fe / H] :=
#   log10((mass_iron / mass_hydrogen)_particle / (mass_iron / mass_hydrogen)_sun)
# my pipeline assumes solar abundances from Asplund et al 2009

print(part['star'].prop('metallicity.total'))
print(part['star'].prop('metallicity.fe'))

In [ ]:
# also use .prop() to compute simple arithmetic combinations, such as [Mg / Fe]

part['star'].prop('metallicity.mg - metallicity.fe')

In [ ]:
# refer to utilities.basic.constant for assumed solar values (Asplund et al 2009) and other constants

ut.constant.sun_composition

# meta-data about simulation

In [ ]:
# dictionary of useful information about the simulation

part.info

In [ ]:
# dictionary of information about this snapshot's index, scale-factor, redshift, time, lookback-time

part.snapshot

In [ ]:
# dictionary class with information about *all* snapshots that were saved for the simulation

print(part.Snapshot.keys())
print(part.Snapshot['redshift'][:10])

In [ ]:
# dictionary class of cosmological parameters, with internal functions for cosmological conversions
# see utilities.cosmology for more on this

part.Cosmology

See gizmo.analysis for examples of high-level analysis, including plotting these data.

See ut.particle for mid-level analysis functions that may be useful.

See other modules within utilities for low-level functions that may be useful.

# coordinates of host galaxy/halo and principal axes of stellar disk

If you enable read_snapshots(assign_host_coordinates=True) (which is True by default), then during read-in the code assigns the position and velocity of the host galaxy/halo (using stars for a baryonic simulations and dark matter for a DM-only simulation). The code stores these coordinates in arrays appended to the particle catalog.

Most simulations have a single host galaxy/halo, but some (like ELVIS) contain two (or more). You can control the number of hosts via: read_snapshots(host_number=2) (by deafult, host_number=1).

Once the code assigns the coordinates of each host, it also can compute the principal axes (rotation tensor) of each host's stellar disk. Enable this via: read_snapshots(assign_host_principal_axes=True) (by default, assign_host_principal_axes=False).

In [ ]:
# position [kpc comoving] and velocity [km / s] of the center of each host galaxy
# can store multiple hosts, though usually just one

print(part.host_positions)
print(part.host_velocities)

In [ ]:
# compute and assign principal axes (defined via moment of inertia tensor) of stars during read in as below

part = gizmo.io.Read.read_snapshots(['star', 'dark'], 'redshift', 0, assign_host_principal_axes=True)

In [ ]:
# rotation tensor[s] of the principal axes for each host are stored via

print(part.host_rotation_tensors[0])

In [ ]:
# now you can compute different types of distance of star particles from the center of each host galaxy
# compute 3-D distance from the host center along simulation's default x,y,z cartesian axes [kpc physical]

part['star'].prop('host.distance')

In [ ]:
# add 'total' to compute total (scalar) distance [kpc physical]

part['star'].prop('host.distance.total')

In [ ]:
# add 'principal' to compute 3-D distance aligned with the principal (major, intermediate, minor) axes of the host [kpc physical]

part['star'].prop('host.distance.principal')

In [ ]:
# add 'cylindrical' to compute 3-D distance aligned with the principal axes in cylindrical coordinates
# first value is along the major axes (R, positive definite)
# second value is vertical height wrt the disk (Z, signed)
# third value is angle (phi, 0 to 2 * pi)

part['star'].prop('host.distance.principal.cylindrical')

In [ ]:
# same for velocity
# compute 3-D velocity from host galaxy center along simulation's default x,y,z cartesian axes [km / s]

part['star'].prop('host.velocity')

In [ ]:
# compute total (scalar) velocity [km / s]

part['star'].prop('host.velocity.total')

In [ ]:
# compute 3-D velocity along the principal (major, intermediate, minor) axes [km / s]

part['star'].prop('host.velocity.principal')

In [ ]:
# compute 3-D velocity in cylindrical coordinates
# first value is along the major axes (positive definite)
# second value is vertical velocity wrt the disk (signed)
# third value is azimuthal velocity in the plane of the disk (positive definite)

part['star'].prop('host.velocity.principal.cylindrical')

In [ ]:
# if you want to store multiple hosts (such as for the ELVIS LG-like paired simulations), 
# set host_number=2 during read-in
# (you can do this for any simulation, it simply finds the second most massive host halo in the zoom-in region)

part = gizmo.io.Read.read_snapshots(['star', 'dark'], 'redshift', 0, host_number=2, assign_host_principal_axes=True)

In [ ]:
# everything above carries over, just use 'host', 'host2', 'host3', etc 
# to identify which host you want coordinates relative to

print(part['star'].prop('host.distance'))
print(part['star'].prop('host2.distance'))

In [ ]:
# the code stores coordinates and rotation tensors for each host

print(part.host_positions)
print(part.host_velocities)
print(part.host_rotation_tensors)

# star particle tracking

Some simulations have pre-compiled HDF5 files to help with tracking star particles over time. These are stored in the directory 'track/' (if present). gizmo_track.py contains the code that generates and reads these files.

star\_indices\_*.hdf5 files store, for each star particles at z = 0, a pointer to where it was in the catalog at each previous snapshot (replace * with snapshot index). This makes it easy to quickly get the properties of a given star particle at any previous snapshot. These pointers are stored in an HDF5 file, one for each previous snapshot.

In [ ]:
# first, read catalog of star particles at z = 0

part_at_z0 = gizmo.io.Read.read_snapshots(['star'], 'redshift', 0)

In [ ]:
# say that you want to find out what they were doing at z = 1
# read in catalog of star particles at z = 1 (snapshot 277)

part_at_z1 = gizmo.io.Read.read_snapshots(['star'], 'redshift', 1)

In [ ]:
# use the function within gizmo_track.py to read star index pointers associated with the catalog z = 1

gizmo.track.ParticleIndexPointer.io_pointers(part_at_z1)

In [ ]:
# pointers are stored via numpy array appended to particle dictionary at the relevant snapshot
# a negative value means that the star formed after this snapshot (so it does not exist at this snapshot)

part_at_z1.index_pointers

In [ ]:
# so, say that you have a list of the indices of star particles of interest at z = 0

indices_at_z0 = np.array([2, 5, 8, 13])

In [ ]:
# their positions at z = 0

part_at_z0['star']['position'][indices_at_z0]

In [ ]:
# get their indices in the catalog at z = 1

indices_at_z1 = part_at_z1.index_pointers[indices_at_z0]
print(indices_at_z1)

In [ ]:
# now you easily can get any property of interest at z = 1, for example, positions

part_at_z1['star']['position'][indices_at_z1]

# star particle formation coordinates

Another part of star particle tracking is storing the position and velocity of each star particle immediately after it forms.

Within track/, star\_form\_coordinates\_600.hdf5 stores, for each star particle at z = 0, its 3-D distance and 3-D velocity wrt to the main host galaxy at the first snapshot after it formed. These coordinates are aligned with the principal (major, intermediate, minor) axes of the stellar disk (as defined via its moment of inertia tensor) at that snapshot.

In [ ]:
# use the function within gizmo_track.py to read this file and assign values directly to the catalog at z = 0

gizmo.track.ParticleCoordinate.io_formation_coordinates(part_at_z0)

In [ ]:
# more conveniently, you can read the formation coordinates of star particles during snapshot read-in

part_at_z0 = gizmo.io.Read.read_snapshots(
    ['star'], 'redshift', 0, assign_host_principal_axes=True, assign_formation_coordinates=True)

In [ ]:
# 3-D distance at formation
# this is aligned with the principal axes of the host galaxy at that time [kpc physical]
# the principal axes are defined *independently* at each snapshot
# distance along dimension 0 is aligned with the major axis
# distance along dimension 1 is algined with the intermediate axis
# distance along dimension 2 is aligned with the minor (Z) axis

part_at_z0['star']['form.host.distance']

In [ ]:
# as before, add 'total' to get the total scalar (absolute) distance wrt the host galaxy at formation [kpc physical]
# this is a derived quantity, so need to call via .prop()

part_at_z0['star'].prop('form.host.distance.total')

In [ ]:
# add 'cylindrical' to get 3-D distance at formation wrt the host galaxy in cylindrical coordinates [kpc physical]

part_at_z0['star'].prop('form.host.distance.cylindrical')

In [ ]:
# these values look more reasonable if you restrict to star particles that formed within the host galaxy

# select particles formed at d = 0 - 8 kpc physical
part_indices = ut.array.get_indices(part_at_z0['star'].prop('form.host.distance.total'), [0, 8])

part_at_z0['star'].prop('form.host.distance.cylindrical', part_indices)

In [ ]:
# same thing for velocity at formation

print(part_at_z0['star']['form.host.velocity'])
print(part_at_z0['star'].prop('form.host.velocity.total'))
print(part_at_z0['star'].prop('form.host.velocity.cylindrical'))

In [ ]:
# recall that formation postion + velocity as stored relative to each host's principal axes
# and that principal axes are computed independently at each snapshot
# the tracking code also stores each host's rotation tensor (for its principal axes) at each snapshot
# so you can use this to compute formation coordinates in the box's x,y,z coordinates if you want

part_at_z0['star'].host_rotation_tensors_at_snapshots

# profile of properties

A common task that you might have is to compute a radial profile of a given quantity, such as mass density, average age, median metallicity, etc.

The high-level functions below make this easier to do.

In [ ]:
# first, initiate an instance of SpeciesProfileClass
# as you initialize,choose your distance/radius binning scheme: 
#   'log' v 'linear', distance limits, bin width, number of spatial dimensions of profile
# refer to ut.binning.DistanceBinClass() for more

# linear binning from 0 to 20 kpc with 1 kpc bin width, assuming a 3-D profile
SpeciesProfile = ut.particle.SpeciesProfileClass(scaling='linear', limits=[0, 20], width=1, dimension_number=3)

In [ ]:
# using this binning scheme,
# compute sum/histogram/density of mass of star particles in each bin
# this returns a bunch of summed properties via a dictionary

pro = SpeciesProfile.get_sum_profiles(part, 'star', 'mass')

In [ ]:
# in principle, you can supply a list of multiple species, and it will compute profiles for each
# thus, it returns a dictionary for each species

pro.keys()

In [ ]:
# the quantities that it stores in each bin

pro['star'].keys()

In [ ]:
# alternately, you may want to compute profiles along a disks R or Z axes
# if so, first define the dimensionality of the profile when you initiate the class

# log binning from 0.1 to 10 kpc with 0.1 dex bin width, assuming a 2-D profile (along R)
SpeciesProfile = ut.particle.SpeciesProfileClass(scaling='log', limits=[0.1, 10], width=0.1, dimension_number=2)

In [ ]:
# set rotation = True to force it to compute profiles along the principal axes (assuming that you read them in)
# use other_axis_distance_limits to limit the extent along the other axis, 
#   in this case, limit the Z axis to within +/- 1 kpc (in the profile, all distances are absolute)

pro = SpeciesProfile.get_sum_profiles(part, 'star', 'mass', rotation=True, other_axis_distance_limits=[0, 1])

In [ ]:
# similarly, do this to compute profiles along Z

# log binning from 0.1 to 10 kpc with 0.1 dex bin width, assuming a 1-D profile (along Z)
SpeciesProfile = ut.particle.SpeciesProfileClass(scaling='log', limits=[0.1, 10], width=0.1, dimension_number=1)

# limit the R axex to [5, 8] kpc 
pro = SpeciesProfile.get_sum_profiles(part, 'star', 'mass', rotation=True, other_axis_distance_limits=[5, 8])

In [ ]:
# using the same binning scheme
# this function computes various statistics of a property of star particles in each bin
# by default, it weights the property by the mass of each particle
# this returns a bunch of statistics via a dictionary

pro = SpeciesProfile.get_statistics_profiles(
    part, 'star', 'age', weight_by_mass=True, rotation=True, other_axis_distance_limits=[5, 8])

In [ ]:
# the quantities that it stores in each bin

pro['star'].keys()